In [1]:
import pandas as pd
df= pd.read_csv(r"C:\Users\shaha\Desktop\Task\data.xlsx - internship-data-1.csv")

In [2]:
df.describe()

,time,Cyclone_Inlet_Gas_Temp,Cyclone_Material_Temp,Cyclone_Outlet_Gas_draft,Cyclone_cone_draft,Cyclone_Gas_Outlet_Temp,Cyclone_Inlet_Draft
count,377719,377719,377719,377719,377719,377719,377719
unique,377719,39736,39662,27669,26487,48044,24010
top,1/15/2018 23:05,Not Connect,0,Not Connect,Not Connect,28.88,Not Connect
freq,1,723,14226,723,723,2036,723


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377719 entries, 0 to 377718
Data columns (total 7 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   time                      377719 non-null  object
 1   Cyclone_Inlet_Gas_Temp    377719 non-null  object
 2   Cyclone_Material_Temp     377719 non-null  object
 3   Cyclone_Outlet_Gas_draft  377719 non-null  object
 4   Cyclone_cone_draft        377719 non-null  object
 5   Cyclone_Gas_Outlet_Temp   377719 non-null  object
 6   Cyclone_Inlet_Draft       377719 non-null  object
dtypes: object(7)
memory usage: 20.2+ MB


In [4]:
df.loc[2472]

time                        1/9/2017 14:00
Cyclone_Inlet_Gas_Temp         I/O Timeout
Cyclone_Material_Temp          I/O Timeout
Cyclone_Outlet_Gas_draft       I/O Timeout
Cyclone_cone_draft             I/O Timeout
Cyclone_Gas_Outlet_Temp        I/O Timeout
Cyclone_Inlet_Draft            I/O Timeout
Name: 2472, dtype: object

In [5]:
df["Temperature of Hot gas leaving the cyclone"] = pd.to_numeric(df["Cyclone_Gas_Outlet_Temp"],errors='coerce')
df["Pressure of gas at inlet of cyclone"] = pd.to_numeric(df["Cyclone_Inlet_Draft"],errors='coerce')
df["Pressure of gas at cone section of cyclone"] = pd.to_numeric(df["Cyclone_cone_draft"],errors='coerce')
df["Pressure of gas at outlet of cyclone"] = pd.to_numeric(df["Cyclone_Outlet_Gas_draft"],errors='coerce')
df["Temperature of Hot gas entering the cyclone"] = pd.to_numeric(df["Cyclone_Inlet_Gas_Temp"],errors='coerce')
df["Temperature of the material at the outlet of the cyclone"] = pd.to_numeric(df["Cyclone_Material_Temp"],errors='coerce')
df['time'] = pd.to_datetime(df['time'])
df=df.drop(columns=['Cyclone_Inlet_Gas_Temp','Cyclone_Material_Temp','Cyclone_Outlet_Gas_draft','Cyclone_cone_draft',
                'Cyclone_Gas_Outlet_Temp','Cyclone_Inlet_Draft'])


In [7]:
df = df.dropna()
df=df.reset_index(drop=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376124 entries, 0 to 376123
Data columns (total 7 columns):
 #   Column                                                    Non-Null Count   Dtype         
---  ------                                                    --------------   -----         
 0   time                                                      376124 non-null  datetime64[ns]
 1   Temperature of Hot gas leaving the cyclone                376124 non-null  float64       
 2   Pressure of gas at inlet of cyclone                       376124 non-null  float64       
 3   Pressure of gas at cone section of cyclone                376124 non-null  float64       
 4   Pressure of gas at outlet of cyclone                      376124 non-null  float64       
 5   Temperature of Hot gas entering the cyclone               376124 non-null  float64       
 6   Temperature of the material at the outlet of the cyclone  376124 non-null  float64       
dtypes: datetime64[ns](1), float64(

In [16]:
df['Date'] = pd.to_datetime(df['time']).dt.date
df=df.drop('time',axis=1)
df['Date']=pd.to_datetime(df['Date'])
df=df.groupby(by='Date').mean()
df.reset_index(level=0, inplace=True)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1314 entries, 0 to 1313
Data columns (total 8 columns):
 #   Column                                                    Non-Null Count  Dtype         
---  ------                                                    --------------  -----         
 0   Date                                                      1314 non-null   datetime64[ns]
 1   Temperature of Hot gas leaving the cyclone                1314 non-null   float64       
 2   Pressure of gas at inlet of cyclone                       1314 non-null   float64       
 3   Pressure of gas at cone section of cyclone                1314 non-null   float64       
 4   Pressure of gas at outlet of cyclone                      1314 non-null   float64       
 5   Temperature of Hot gas entering the cyclone               1314 non-null   float64       
 6   Temperature of the material at the outlet of the cyclone  1314 non-null   float64       
 7   outliers                                   

In [19]:
from sklearn.ensemble import IsolationForest
import plotly.express as px
model =  IsolationForest(contamination=0.1)
model.fit(df[['Temperature of Hot gas leaving the cyclone']])
df['outliers']=pd.Series(model.predict(df[['Temperature of Hot gas leaving the cyclone']])).apply(lambda x: 'yes' if (x == -1) else 'no' )
df1=df.query('outliers=="yes"')
df1.to_csv(r"C:\Users\shaha\Desktop\Task\leaving_cyclone.csv")
fig = px.scatter(df.reset_index(), x='Date', y='Temperature of Hot gas leaving the cyclone', color='outliers')
fig.update_xaxes(rangeslider_visible=True,)
fig.show()

In [20]:
from sklearn.ensemble import IsolationForest
import plotly.express as px
model =  IsolationForest(contamination=0.1)
model.fit(df[['Temperature of Hot gas entering the cyclone']])
df['outliers']=pd.Series(model.predict(df[['Temperature of Hot gas entering the cyclone']])).apply(lambda x: 'yes' if (x == -1) else 'no' )
df2=df.query('outliers=="yes"')
df2.to_csv(r"C:\Users\shaha\Desktop\Task\entering_cyclone.csv")
fig = px.scatter(df.reset_index(), x='Date', y='Temperature of Hot gas entering the cyclone', color='outliers')
fig.update_xaxes(rangeslider_visible=True,)
fig.show()

In [21]:
from sklearn.ensemble import IsolationForest
import plotly.express as px
model =  IsolationForest(contamination=0.1)
model.fit(df[['Pressure of gas at outlet of cyclone']])
df['outliers']=pd.Series(model.predict(df[['Pressure of gas at outlet of cyclone']])).apply(lambda x: 'yes' if (x == -1) else 'no' )
df3=df.query('outliers=="yes"')
df3.to_csv(r"C:\Users\shaha\Desktop\Task\Pressure_outlet.csv")
fig = px.scatter(df.reset_index(), x='Date', y='Pressure of gas at outlet of cyclone', color='outliers')
fig.update_xaxes(rangeslider_visible=True,)
fig.show()

In [22]:
from sklearn.ensemble import IsolationForest
import plotly.express as px
model =  IsolationForest(contamination=0.1)
model.fit(df[['Pressure of gas at cone section of cyclone']])
df['outliers']=pd.Series(model.predict(df[['Pressure of gas at cone section of cyclone']])).apply(lambda x: 'yes' if (x == -1) else 'no' )
df4=df.query('outliers=="yes"')
df4.to_csv(r"C:\Users\shaha\Desktop\Task\Pressure_cone.csv")
fig = px.scatter(df.reset_index(), x='Date', y='Pressure of gas at cone section of cyclone', color='outliers')
fig.update_xaxes(rangeslider_visible=True,)
fig.show()

In [23]:
from sklearn.ensemble import IsolationForest
import plotly.express as px
model =  IsolationForest(contamination=0.1)
model.fit(df[['Pressure of gas at inlet of cyclone']])
df['outliers']=pd.Series(model.predict(df[['Pressure of gas at inlet of cyclone']])).apply(lambda x: 'yes' if (x == -1) else 'no' )
df5=df.query('outliers=="yes"')
df5.to_csv(r"C:\Users\shaha\Desktop\Task\Pressure_inlet.csv")
fig = px.scatter(df.reset_index(), x='Date', y='Pressure of gas at inlet of cyclone', color='outliers')
fig.update_xaxes(rangeslider_visible=True,)
fig.show()

In [24]:
from sklearn.ensemble import IsolationForest
import plotly.express as px
model =  IsolationForest(contamination=0.1)
model.fit(df[['Temperature of the material at the outlet of the cyclone']])
df['outliers']=pd.Series(model.predict(df[['Temperature of the material at the outlet of the cyclone']])).apply(lambda x: 'yes' if (x == -1) else 'no' )
df6=df.query('outliers=="yes"')
df6.to_csv(r"C:\Users\shaha\Desktop\Task\meterial_temp.csv")
fig = px.scatter(df.reset_index(), x='Date', y='Temperature of the material at the outlet of the cyclone', color='outliers')
fig.update_xaxes(rangeslider_visible=True,)
fig.show()